# Analyzing CIA Factbook data using SQL

## I will read in a database and then query the database using SQL Lite.  This project will show my basic SQL skills and analysis skills.  Project derived from DataQuest.io as a recommended project for a Portfolio.

### The read in Table will have the following columns with the following meanings
### name — the name of the country.
### area— the country's total area (both land and water).
### area_land — the country's land area in square kilometers.
### area_water — the country's waterarea in square kilometers.
### population — the country's population.
### population_growth— the country's population growth as a percentage.
### birth_rate — the country's birth rate, or the number of births per year per 1,000 people.
### death_rate — the country's death rate, or the number of death per year per 1,000 people.

## Part 2:  Connect to Database

In [111]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

## Part 3:  Get info on Tables within Database, Query first 5 rows

In [112]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [113]:
%%sql
SELECT * FROM facts LIMIT 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


### It appears the rows are sorted in alphabetical order according to country name

## Parts 4 - 6:  Calculate Summary Statistics

### minimum population

In [114]:
%%sql
SELECT name, MIN(population) from facts 

Done.


name,MIN(population)
Antarctica,0


### Minimum population is Antarctica with zero.  Because Antarctica is not really a country, we will have to exclude this later

### Maximum Population

In [115]:
%%sql
SELECT name, MAX(population) from facts 

Done.


name,MAX(population)
World,7256490011


### World is not a country either, this will have to be recalculated also

### Minimum Population Growth

In [116]:
%%sql
select min(population_growth) from facts

Done.


min(population_growth)
0.0


### it looks like this table fails to recite negative population growth

In [117]:
%%sql
select name from facts where population_growth = (select min(population_growth) from facts)

Done.


name
Holy See (Vatican City)
Cocos (Keeling) Islands
Greenland
Pitcairn Islands


### It appears that 4 countries have a zero or minimum population growth

### Maximum Population Growth

In [118]:
%%sql
SELECT name, MAX(population_growth) from facts 

Done.


name,MAX(population_growth)
South Sudan,4.02


### List of countries with lowest population

In [119]:
%%sql
select name, population from facts order by population limit 5

Done.


name,population
Ashmore and Cartier Islands,None
Coral Sea Islands,None
Heard Island and McDonald Islands,None
Clipperton Island,None
French Southern and Antarctic Lands,None


### That is unfortunate.  We'll have to ignore the null values 

In [120]:
%%sql
select name, population from facts where (population is not null) AND (name <> 'Antarctica') 
order by population limit 10

Done.


name,population
Pitcairn Islands,48
Cocos (Keeling) Islands,596
Holy See (Vatican City),842
Niue,1190
Tokelau,1337
Christmas Island,1530
Svalbard,1872
Norfolk Island,2210
Falkland Islands (Islas Malvinas),3361
Montserrat,5241


### now we know country names of least populous countries.  Most of these are islands or archipelagos

### List of countries with most population excluding 'World' and 'European Union'

In [121]:
%%sql
select name, population from facts where name not in 
('World', 'European Union') order by population desc limit 10

Done.


name,population
China,1367485388
India,1251695584
United States,321368864
Indonesia,255993674
Brazil,204259812
Pakistan,199085847
Nigeria,181562056
Bangladesh,168957745
Russia,142423773
Japan,126919659


### China is No. 1, and U.S. is No. 3!

### Calculate Average Population from Facts table

In [122]:
%%sql
select avg(population) from facts

Done.


avg(population)
62094928.32231405


### Calculate average Area from Facts table

In [123]:
%%sql
select avg(area) from facts

Done.


avg(area)
555093.546184739


## Part 7:  Countries with population above average AND area below average

In [133]:
%%sql
SELECT name, round(population/1000000.0, 2) as pop_in_millions, area FROM facts WHERE population > (SELECT AVG(population) FROM facts) AND 
area < (SELECT AVG(area) FROM facts);

Done.


name,pop_in_millions,area
Bangladesh,168.96,148460
Germany,80.85,357022
Japan,126.92,377915
Philippines,101.0,300000
Thailand,67.98,513120
United Kingdom,64.09,243610
Vietnam,94.35,331210


## Part 8:  Lets explore other statistics


### Which countries have the highest ratios of water area to land area?

In [125]:
%%sql
SELECT name, round(area_water/cast (area_land as float), 3) as ratio 
FROM facts order by ratio desc limit 10

Done.


name,ratio
British Indian Ocean Territory,905.667
Virgin Islands,4.52
Puerto Rico,0.555
"Bahamas, The",0.387
Guinea-Bissau,0.285
Malawi,0.259
Netherlands,0.226
Uganda,0.223
Eritrea,0.164
Liberia,0.156


### Which countries have more water area than land area?

In [126]:
%%sql
select name, area_water, area_land from facts where area_water > area_land

Done.


name,area_water,area_land
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346


### Which countries will add the most people in the next year?

In [127]:
%%sql
select name, round(population_growth * population/100000000, 2) as inc_in_pop_millions from facts WHERE name <> 'World'
order by inc_in_pop_millions desc limit 10

Done.


name,inc_in_pop_millions
India,15.27
China,6.15
Nigeria,4.45
Pakistan,2.91
Ethiopia,2.87
Bangladesh,2.7
United States,2.51
Indonesia,2.36
"Congo, Democratic Republic of the",1.94
Philippines,1.63


### increase in population is the product of population_growth and population. A factor of 100 is included for percentage. India beats out China on this metric!

### Countries where death rate exceeds birth rate?

In [128]:
%%sql
select name, birth_rate, death_rate, round(death_rate-birth_rate, 2) as diff from facts WHERE death_rate > birth_rate order by diff desc

Done.


name,birth_rate,death_rate,diff
Bulgaria,8.92,14.44,5.52
Serbia,9.08,13.66,4.58
Latvia,10.0,14.31,4.31
Lithuania,10.1,14.27,4.17
Ukraine,10.72,14.46,3.74
Hungary,9.16,12.73,3.57
Germany,8.47,11.42,2.95
Slovenia,8.42,11.37,2.95
Romania,9.14,11.9,2.76
Croatia,9.45,12.18,2.73


### It appears that Eastern European countries have death rates greatly exceeding the birth rates!

### Countries with highest population/area ratio (i.e. population density)?

In [135]:
%%sql
select name, round(population/1000000.0, 2) as pop_in_millions, area, population/area as pop_density from facts order by pop_density desc limit 10

Done.


name,pop_in_millions,area,pop_density
Macau,0.59,28,21168
Monaco,0.03,2,15267
Singapore,5.67,697,8141
Hong Kong,7.14,1108,6445
Gaza Strip,1.87,360,5191
Gibraltar,0.03,6,4876
Bahrain,1.35,760,1771
Maldives,0.39,298,1319
Malta,0.41,316,1310
Bermuda,0.07,54,1299


### Its those tiny city states that have the greatest population density
### How does this list compare to previous list of countries with above average population and below average area?  Answer = no overlap.  None of that list showed up in the top 10 here.  This is because the above city states do not have a population that is high enough so that it is above average.